In [2]:
%env GOOGLE_APPLICATION_CREDENTIALS=/usr/local/creds/GoogleDev-f20867bbe55e.json

env: GOOGLE_APPLICATION_CREDENTIALS=/usr/local/creds/GoogleDev-f20867bbe55e.json


In [1]:
import logging
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
        
def print_info(s):
    print s
    
def run_pipeline():
    # Prepare necessary options for beam pipeline
    # https://beam.apache.org/documentation/runners/capability-matrix/
    #
    # Pipline parameters in detail
    # https://cloud.google.com/dataflow/pipelines/specifying-exec-params
    
    # Specify the which runner to use
    # RUNNER          = "DirectRunner"
    RUNNER            = "DataflowRunner"
    
    # Path to store the code package used by worker in during staging
    STAGING_LOCATION  = "gs://myfirstbucket-1/staging"
    
    # Path to store temporary jobfile used by worker node in pipeline execution
    TEMP_LOCATION     = "gs://myfirstbucket-1/tmp"
    
    # Job name for this session
    # * Used when gcp dataflow is used as runner
    JOB_NAME          = "job-write-file"

    # Project name in GCP. Needed when Data Flow Runner is used 
    # * Used when gcp dataflow is used as runner
    PROJECT           = "elite-caster-125113"
    
    # File path of file to read from
    #FILE_PATH        = "/home/ywatanabe/notebook/sample.txt"
    FILE_PATH         = "gs://ywatanabe/Documents/sample.txt"
    
    # Region name where pipeline to operate
    REGION            = "asia-northeast1"
    
    options= {
        'project'          : PROJECT,
        'runner'           : RUNNER,
        'staging_location' : STAGING_LOCATION,
        'temp_location'    : TEMP_LOCATION,
        'job_name'         : JOB_NAME,
        "region"           : REGION
    }

    # Configure Pipeline by creating PiplineOptions() object 
    o = beam.pipeline.PipelineOptions( **options )
    
    # Build Pipeline
    with beam.Pipeline( options = o ) as p:
        logging.info('Pipeline created.')
        # Create PCollection object
        # Read lines from file
        lines = p | 'ReadFromText' >> ReadFromText( FILE_PATH )
    
        debug = ( lines | 'Print' >> beam.Map(print_info) )
    
        # Just simply write out the lines
        lines | 'WriteToText' >>  WriteToText(
            FILE_PATH, file_name_suffix='-write'
        )

def main():
    run_pipeline()
    
if __name__ == '__main__':
    logging.getLogger().setLevel(logging.INFO)
    main()

INFO:root:Pipeline created.
/opt/anaconda2/lib/python2.7/site-packages/apache_beam/io/gcp/gcsio.py:166: DeprecationWarning: object() takes no parameters
  super(GcsIO, cls).__new__(cls, storage_client))
INFO:root:Starting the size estimation of the input
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:root:Finished the size estimation of the input at 1 files. Estimation took 0.438647985458 seconds
INFO:root:Starting the size estimation of the input
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:root:Finished the size estimation of the input at 1 files. Estimation took 0.390017986298 seconds
INFO:root:Starting GCS upload to gs://myfirstbucket-1/staging/job-write-file.1527554868.768140/pipeline.pb...
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_to

INFO:root:2018-05-29T00:49:41.135Z: JOB_MESSAGE_BASIC: Executing operation WriteToText/Write/WriteImpl/GroupByKey/Read+WriteToText/Write/WriteImpl/GroupByKey/GroupByWindow+WriteToText/Write/WriteImpl/Extract
INFO:root:2018-05-29T00:49:52.036Z: JOB_MESSAGE_DEBUG: Value "WriteToText/Write/WriteImpl/Extract.out" materialized.
INFO:root:2018-05-29T00:49:52.050Z: JOB_MESSAGE_BASIC: Executing operation WriteToText/Write/WriteImpl/FinalizeWrite/_UnpickledSideInput(Extract.out.0)
INFO:root:2018-05-29T00:49:52.213Z: JOB_MESSAGE_DEBUG: Value "WriteToText/Write/WriteImpl/FinalizeWrite/_UnpickledSideInput(Extract.out.0).output" materialized.
INFO:root:2018-05-29T00:49:52.227Z: JOB_MESSAGE_BASIC: Executing operation WriteToText/Write/WriteImpl/FinalizeWrite/FinalizeWrite
INFO:root:2018-05-29T00:49:59.302Z: JOB_MESSAGE_DEBUG: Executing success step success13
INFO:root:2018-05-29T00:50:00.101Z: JOB_MESSAGE_DETAILED: Cleaning up.
INFO:root:2018-05-29T00:50:00.326Z: JOB_MESSAGE_DEBUG: Starting worker p